After exploration is done, it is time to use the knowledge obtained there.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter, defaultdict
from math import hypot
from IPython.display import display, HTML, clear_output

In [2]:
df_app_events               = pd.DataFrame.from_csv("data/app_events.csv")
df_app_labels               = pd.DataFrame.from_csv("data/app_labels.csv")
df_events                   = pd.DataFrame.from_csv("data/events.csv")
df_gender_age_train         = pd.DataFrame.from_csv("data/gender_age_train.csv")
df_label_categories         = pd.DataFrame.from_csv("data/label_categories.csv")
df_phone_brand_device_model = pd.DataFrame.from_csv("data/phone_brand_device_model.csv")

df_gender_age_test          = pd.DataFrame.from_csv("data/gender_age_test.csv")

In [3]:
df_app_events.drop(['is_installed'], axis=1, inplace=True)

In [4]:
%%time
def get_day(dt):
    return datetime.strptime(dt, '%Y-%m-%d %H:%M:%S').weekday()

def get_hour(dt):
    return datetime.strptime(dt, '%Y-%m-%d %H:%M:%S').hour

df_events['week_day'] = df_events['timestamp'].apply(get_day)
df_events['hour']     = df_events['timestamp'].apply(get_hour)

df_events.drop(['timestamp'], axis=1, inplace=True)

CPU times: user 2min 51s, sys: 1.38 s, total: 2min 52s
Wall time: 3min 1s


In [5]:
brands_map = {
    "三星": "samsung","天语": "ktouch","海信": "hisense","联想": "lenovo","欧比": "obi","爱派尔": "ipair",
    "努比亚": "nubia","优米": "youmi","朵唯": "dowe","黑米": "heymi","锤子": "hammer","酷比魔方": "koobee","美图": "meitu",
    "尼比鲁": "nibilu","一加": "oneplus","优购": "yougo","诺基亚": "nokia","糖葫芦": "candy","中国移动": "ccmc","语信": "yuxin",
    "基伍": "kiwu","青橙": "greeno","华硕": "asus","夏新": "panosonic","维图": "weitu","艾优尼": "aiyouni","摩托罗拉": "moto",
    "乡米": "xiangmi","米奇": "micky","大可乐": "bigcola","沃普丰": "wpf","神舟": "hasse","摩乐": "mole","飞秒": "fs",
    "米歌": "mige","富可视": "fks","德赛": "desci","梦米": "mengmi","乐视": "lshi","小杨树": "smallt","纽曼": "newman",
    "邦华": "banghua","E派": "epai","易派": "epai","普耐尔": "pner","欧新": "ouxin","西米": "ximi","海尔": "haier",
    "波导": "bodao","糯米": "nuomi","唯米": "weimi","酷珀": "kupo","谷歌": "google","昂达": "ada","聆韵": "lingyun",
    "小米": "xiaomi","华为": "Huawei","魅族": "Meizu","中兴": "ZTE","酷派": "coolpad","金立": "gionee","SUGAR": "sugar",
    "OPPO": "oppo","vivo": "vivo","HTC": "htc","LG": "lg","ZUK": "zuk","TCL": "tcl","LOGO": "logo","Lovme": "lovme",
    "PPTV": "pptv","ZOYE": "zoye","MIL": "mil","索尼" : "sony","欧博信" : "opssom","奇酷" : "qiku","酷比" : "cube",
    "康佳" : "konka","亿通" : "yitong","金星数码" : "jxd","至尊宝" : "monkey king","百立丰" : "hundred li feng",
    "贝尔丰" : "bifer","百加" : "bacardi","诺亚信" : "noain","广信" : "kingsun","世纪天元" : "ctyon","青葱" : "cong",
    "果米" : "taobao","斐讯" : "phicomm","长虹" : "changhong","欧奇" : "oukimobile","先锋" : "xfplay",
    "台电" : "teclast","大Q" : "daq","蓝魔" : "ramos","奥克斯" : "aux","飞利浦": "philips","智镁": "zhimei",
    "惠普": "hp","原点": "origin","戴尔": "dell","碟米": "diemi","西门子": "siemens","亚马逊": "amazon","宏碁": "acer"
}

In [6]:
df_phone_brand_device_model['phone_brand'] = df_phone_brand_device_model['phone_brand'].apply(lambda x: brands_map.get(x, 'other'))
df_phone_brand_device_model = df_phone_brand_device_model.groupby(df_phone_brand_device_model.index).first()

After two hours of manual types selection (was aware of [this](https://www.kaggle.com/nickdudaev/talkingdata-mobile-user-demographics/categorizing-labels/comments), but didn't like it), I created the following categories.

In [7]:
category_types = {
    'game': 1,                  'art': 2,
    'lifestyle': 3,             'strategy': 4,              'pet': 5,
    'sport': 6,                 'psychology': 7,            'business': 8,
    'child': 9,                 'learning': 10,             'love': 11,
    'entertainment': 12,        'books': 13,                'shopping': 14,
    'school': 15,               'photo': 16,                'messangers': 17,
    'travel': 18,               'utilities': 19,            'food': 20,
    'productivity': 21,         'car': 22,                  'clothes': 23,
    'beauty': 24,               'woman': 25,                'man': 26,
    'home': 27,                 'luxury': 28,               'health': 29
}
list_types = {
    'game-game type': [1],
    'game-Game themes': [1],
    'game-Art Style': [1, 2],
    'game-Leisure time': [1, 3],
    'game-Cutting things': [1, 4],
    'game-Finding fault': [1, 4],
    'game-stress reliever': [1],
    'game-pet': [1, 5],
    'game-Answer': [1],
    'game-Fishing': [1, 3],
    'game-Music and Dance': [1, 2],
    'game-Puzzle': [1, 4],
    'game-Adventure puzzle': [1, 4],
    'game-Parkour Racing': [1, 6],
    'game-Parkour': [1, 6],
    'game-Racing': [1, 6],
    'game-Motorcycle': [1, 6],
    'game-Rowing': [1, 6],
    'game-aircraft': [1, 6],
    'game-Gem Elimination': [1],
    'game-Box': [1, 6],
    'game-gem': [1],
    'game-Lianliankan': [1],
    'game-Zuma': [1],
    'game-Bobble': [1],
    'game-Cartoon': [1],
    'game-Online game': [1],
    'game-Role -playing games': [1, 7],
    'game-Competitive action': [1, 4],
    'game-Tactics': [1, 4],
    'game-card': [1, 4],
    'game-Business simulation': [1, 8],
    'game-Action Shooting': [1, 6],
    'game-Horizontal version': [1],
    'game-shooting': [1, 6],
    'game-3D': [1],
    'game-flight': [1, 6],
    'game-tank': [1],
    'game-Snipe': [1],
    'poker and chess': [1, 4],
    'poker game_doudizhu': [1, 4],
    'chess': [1, 4],
    'majiang': [1, 4],
    'table game': [1],
    'Texas Poker': [1, 7],
    'poker': [1, 7],
    'Children puzzle game': [1, 9],
    'Puzzle': [1, 4],
    'Literacy Games': [9, 10],
    'Intellectual development game': [1, 4],
    'math': [10],
    'Guard tower defense game': [1, 4],
    'checkpoints game': [1],
    'Beach landing game': [1],
    'fighting game': [1, 6],
    'old style game': [1],
    'basketball': [1, 6],
    'football': [1, 6],
    'tennis': [1, 6],
    'billards': [1, 6],
    'other ball game': [1, 6],
    'RPG game': [1, 4],
    'Turn based RPG game': [1, 4],
    'realtime fighting': [1, 6],
    'ARPG': [1, 4],
    'business strategy game': [1, 4, 8],
    'war chess': [1, 4],
    'business': [8],
    'farm': [1, 3],
    'raising up game': [1, 9],
    'pet raising up': [1, 9, 5],
    'love raising up': [11],
    'unknown': [],
    'puzzel': [1, 4],
    'gambling': [1, 4, 7],
    '3 kindom game': [1, 4],
    'knight game': [1, 4],
    'Cultivation fantasy game': [1, 7],
    'Journey to the West game': [1],
    'Outlaws of the Marsh game': [1],
    'Chinese Classical Mythology': [10],
    'Western Mythology': [10],
    '80s Japanese comic': [12],
    '90s Japanese comic': [12],
    'movie': [12],
    'shows': [12],
    'millitary and wars': [1, 4],
    'science fiction': [13],
    'zombies game': [1],
    'World of Warcraft': [1, 4],
    'magic': [3],
    'dotal-lol': [1, 4],
    'sailing game': [1, 6, 3],
    'sports': [6],
    'Chinese painting': [2, 3],
    'japanese and korean style': [2, 3],
    'japanese comic and animation': [12],
    'US and Europe animation': [12],
    'Cute style comic': [12],
    'pixel style comic': [12],
    'violence comic': [12],
    'US and Europe magic comic': [12],
    'realistic style comic': [12],
    'chinese comic': [12],
    'online shopping': [14],
    'online malls': [14],
    'online shopping by group, like groupon': [14],
    'online shopping navigation': [14],
    'online shopping, price comparing': [14],
    'shopping sharing': [14],
    'education': [10],
    'education for babies': [10, 9],
    'education outside class': [10],
    'study abroad': [10],
    'foreign language': [10],
    'professional skills': [10],
    'management': [8],
    'art and culture': [2, 3],
    'library': [10],
    'exams': [15],
    'dialects': [10],
    'class managemetn': [8],
    'reading': [13],
    'reading for fetus': [13, 9],
    'children books': [13, 9],
    'novels': [13],
    'magazine and journal': [13, 12],
    'comics': [12],
    'dictionary': [10],
    'reading platform': [10, 13],
    'poetry': [2, 3],
    'joke': [12],
    'psychology': [7],
    'science': [10],
    'Information': [10, 13],
    'news': [12],
    'Entertainment News': [12],
    'Academic Information': [10],
    'Sports News': [12, 6],
    'Financial Information': [8],
    'Technology Information': [10],
    'gaming strategy': [1, 4],
    'Socially': [7],
    'weibo': [12],
    'love and marriage': [11],
    'community': [7],
    'picture sharing': [16],
    'blogs': [12],
    'communitation': [17],
    'IM': [17],
    'mesasge': [17],
    'email': [17],
    'phone': [17],
    'movie': [12],
    'radio': [12],
    'music': [12],
    'video': [12],
    'show': [12],
    'Audiobooks': [12, 13],
    'Business Travel': [18, 8],
    'map': [18],
    'navigation': [18],
    'Behalf of the drive': [18],
    'Taxi': [18],
    'Car': [18],
    'flight': [18],
    'Bus': [18],
    'train': [18],
    'Hotel application': [18],
    'Travel advisory': [18],
    'tourism product': [18],
    'Share Travels': [18],
    'coach transport': [18],
    'Household': [3, 19],
    'Decoration': [3, 2, 19],
    'Appliances': [19],
    'Furniture': [19],
    'household products': [19],
    'Smart Home': [19],
    'health': [6],
    'lose weight': [6],
    'sports and gym': [6],
    'health': [6],
    'Medical': [6],
    'Health Management': [6],
    'Living': [3],
    'pictures photography': [16],
    'Man playing favorites': [1],
    'takeaway ordering': [20],
    'Astrology Horoscope': [7],
    'coupon': [14],
    'Reviews' : [14],
    'Recipes': [20],
    'Business Office': [8],
    'Weather': [18],
    'Clock': [21],
    'Express': [18, 14],
    'Calendar': [21],
    'Accounting': [8],
    'Car Owners': [22],
    'convenience services': [14],
    'fashion outfit': [23, 2],
    'Points Activities': [3],
    'Housekeeping': [19],
    'Integrated Living': [19],
    'work': [8],
    'Jobs': [8],
    'Scheduling': [21],
    'notes': [21],
    'File Editor': [21],
    'business cards': [8],
    'Contacts': [8, 21],
    'network disk': [19],
    'store management': [21],
    'Engineering Drawing': [21],
    'phone tools': [21],
    'System Tools': [21],
    'App Store': [21],
    'search': [21],
    'Input Method': [],
    'Browser': [21],
    'Desktop Enhancements': [21],
    'readers': [13],
    'Utilities': [19],
    'Editor': [19],
    'WIFI': [21],
    'Finance': [8],
    'Wealth Management': [8],
    'IMF': [8],
    'Direct Bank': [8],
    'Direct Insurance': [8],
    'Securities': [8],
    'futures': [8],
    'Exchange': [8],
    'Precious Metals': [8],
    'Crowdfunding financing': [8],
    'Lottery ticket': [1],
    'Pay': [14],
    'Debit and credit': [8],
    'Estate': [8],
    'Housing Advice': [3],
    'Buy': [14],
    'Sellers': [14],
    'Rentals': [14],
    'Mother': [9],
    'Prepare pregnant pregnancy': [9],
    'Parenting': [9],
    'Maternal and child supplies': [9],
    'entertainment': [12],
    'KTV': [12],
    'show': [12],
    'the film': [12],
    'car': [22],
    'Automotive News': [22, 12],
    'Sale of cars': [14, 22],
    'A beauty care': [24],
    'Skin care applications': [24],
    'Beauty Nail': [24],
    'Make-up application': [24],
    'unknown': [],
    'Consumer preferences': [14],
    'Consumer Goods': [14],
    'Consumer orientation': [14],
    'Brand Country': [14],
    'College Students': [15],
    'Condition of the vehicles': [22],
    'Car': [22],
    'No Car': [22],
    'Jewelry Watches': [24],
    'branded watch': [3],
    'fashion brand': [23, 24],
    'Switzerland': [18],
    'Dress shoes': [23],
    'Women': [25],
    'Shoes': [23],
    "Men's": [26],
    "Men's Shoes": [26],
    'France': [18],
    'Hong Kong': [18],
    'Japan': [18],
    'High-end brand': [23, 24],
    'United States': [18],
    'Outdoor sport': [6],
    'Germany':  [18],
    'underwear': [23],
    'Home Clothing': [23],
    'Volkswagen brand': [22],
    'Sweden': [18],
    'Accessories': [19],
    'Italy': [18],
    'Household utensils': [19],
    'bed linings': [3],
    'Kids': [9],
    'Sports Health': [6],
    'outdoor equipment': [6],
    'China Mainland': [18],
    'Bags': [23],
    'Lady bags': [23],
    'Taiwan': [18],
    'Baby Products': [9, 14],
    'Baby Clothing': [9, 23],
    'M package': [],
    'wallet': [8],
    'Digital': [],
    'camera': [21],
    'Culture and Education': [2, 13],
    'Office Supplies': [21],
    'Jewelry, jewelry': [24],
    'Swimming Supplies': [6],
    'Canada': [18],
    'Phone': [21],
    'Digital Accessories': [19],
    'computer': [19],
    'Luxury brand': [24],
    'cosmetic': [24],
    'Fragrance oils': [24],
    'Feature Pack': [],
    'Suitcase': [8],
    'Makeup': [24],
    'Skin Care': [24],
    'FOOD': [20],
    'Fast food meals': [20],
    'Day feed': [20],
    'Home Decoration': [27],
    'United Kingdom': [18],
    'Collection process': [19],
    'Korea': [18],
    'Austria': [18],
    'Spain': [18],
    'Household appliances': [27, 19],
    'home appliances': [27, 19],
    'Bathroom products': [27],
    'Pregnant mother supplies': [9, 25],
    'Shoes': [23],
    'Norway': [18],
    'Western Dinner': [20],
    'Kitchen Appliances': [27, 19],
    'Australia': [18],
    'Leisure and entertainment': [12],
    "Children's entertainment": [12, 9],
    'SPA': [24],
    'Water Coffee Bar': [20],
    'Domestic services': [27],
    'Nail Tattoos': [24, 25],
    'Custom label': [],
    'DS_P2P net loan': [8],
    'Location': [18],
    'Go places': [18],
    'High places': [18],
    'Four five-star hotel': [18, 28],
    'Golf Club': [6, 26],
    'ski facility': [6],
    'transportation hub': [18],
    'AIRPORT': [18],
    'Culture & Education': [10],
    'the University': [10, 15],
    'High school': [10, 15],
    'Tourist attractions': [18],
    'hospital': [29],
    'Portugal': [18],
    'Large household appliances': [27],
    'Baby food': [20, 9],
    'Snacks Snacks': [20],
    'Retail stores': [14],
    'Tobacco row': [14],
    'Decoration materials': [2],
    'Fitness Equipment': [6],
    'Hotels': [18],
    'Singapore': [18],
    'Korean food': [20],
    'Lunch Dinner': [20],
    'Bread dessert': [20],
    'Thailand': [18],
    'Belgium': [18],
    'Shabu hot pot': [20],
    'buffet': [20],
    'groceries': [20, 14],
    'Convenience Store': [14],
    'Pharmacy / medical devices / Health': [29],
    'Denmark': [18],
    'Greece': [18],
    'Furniture': [27],
    'Kitchenware': [27],
    'Collection shop': [27, 14],
    'Baby Care': [9],
    'Netherlands': [18],
    'Stereo headphones': [14],
    'Baby bed': [27, 9],
    'kids toys': [9],
    'Digital stores': [19],
    'Education and Training': [10],
    'Swim': [6],
    'wig': [],
    'tableware': [27],
    'Latvia': [18],
    'Coffee water bar / teahouse': [20],
    'retail': [14],
    'Brazil': [18],
    'Malaysia': [18],
    'Baby Home Textiles': [27, 9],
    'SPA / Health Center': [29, 24],
    'Finland': [18],
    'Laundry / Leather Care / change clothes': [23],
    'Sports': [6],
    'cinema': [12],
    'Musical Instruments': [2],
    'Beauty salons': [24],
    'Supermarkets': [14, 20],
    'Financial Planning Services': [8],
    'car services': [22],
    'Auto 4S shop': [22],
    'tea shop': [20],
    'Photographic prints': [16],
    'Cycling Sport': [6],
    'Wedding services': [11],
    'KTV service': [12],
    'Appliance stores': [19],
    'Philippines': [18],
    'Tea cups': [27],
    'Board games and entertainment': [1, 12],
    'Games': [1],
    'Gym': [6],
    'Cleaning Products': [14],
    'Experience': [10],
    'Car decoration': [22, 2],
    'Real Estate Property': [8],
    'Housing agency': [8],
    'Florist / Fruit Shop': [2, 25],
    'Fishing Tackle': [3],
    'Electric Car': [22],
    'Personal Care': [24],
    'Swimming pool': [6],
    'Video City': [12],
    'Aftermarket decoration': [2],
    'Billiard room': [6],
    'Foot Bath': [24],
    'Wedding photography': [11, 16],
    'Pet Services': [5],
    'Internet cafes': [19],
    'bank': [8],
    'Parental photography': [9, 16],
    'pharmacy': [29],
    'Supermarket': [20],
    'teahouse': [20],
    'Car Rental': [22],
    'bookstore': [13],
    'Health Products': [29],
    'flower shop': [27, 25],
    'Driving School': [22],
    'Games deep tags':[1],
    'Game willingness to pay':[1],
    'Game amount paid': [1],
    'Game Frequency': [1],
    'Game Installation': [1],
    'Active Games': [1],
    'car brand': [22],
    'Industry tag': [27],
    'Property Industry 1.0': [27],
    'Customization': [19],
    'Irritation / Fun': [1],
    'natural': [1],
    'Science and Technology': [10],
    'vitality': [29],
    'Smart Shopping': [14],
    'Personal Effectiveness': [21],
    'Teahouse': [20],
    'Toiletries': [19],
    'Leather Care': [23],
    'cinema': [12],
    'Musical instruments': [2],
    'Indian Express': [20],
    'Foot Massage': [24],
    'Experience / DIY': [27],
    'Medical': [29],
    'marry': [11],
    'beauty': [24],
    'Hairdressing': [24],
    'financial': [8],
    'Paternity': [26],
    'Nail': [24],
    'Beauty SPA': [24],
    'Early Learning': [9, 10],
    'medical instruments': [29],
    'Training institutions': [10],
    'Exam training': [15, 10],
    'Brands': [14],
    'Phone': [14],
    'Smart TV': [12],
    'smart watch': [12],
    'Smart bracelet': [12],
    'shopping center': [14],
    'language training': [10],
    'Car Accessories': [22],
    'pawn shop': [8],
    'Bath': [24],
    'Express Inn': [18],
    'CAR WASH': [22],
    'General merchandise': [14],
    'Real Estate Agents': [8],
    'securities company': [8],
    'post Office': [19],
    'SCHOOL': [15, 10],
    'the University': [15, 10],
    'dance': [2],
    'Parenting': [9],
    'Month Center': [25, 9],
    'Museum of Traditional Chinese Medicine': [29],
    'Corporate': [8],
    'Travel': [18],
    "Parent-child's play": [9],
    'Personalized photo': [16],
    'Optical Shop': [29],
    'Star Hotels': [18],
    'Logistics companies': [18],
    'Skills Training': [10],
    'Concert hall': [2, 12],
    'Food company': [20],
    'Insurance company': [8],
    'car repair': [22],
    'model': [24],
    'Wearable Health': [29],
    'Smart Appliances': [19],
    'Science and Technology 1': [10],
    '1 vitality': [29],
    'Smart Shopping 1': [14],
    'Personal Effectiveness 1': [21],
    'A shares': [8],
    'B Shares': [8],
    'H shares': [8],
    'N shares': [8],
    'Other shares': [8],
    'fund': [8],
    'Monetary Fund': [8],
    'Mixed Fund': [8],
    'Bond Fund': [8],
    'Capital Preservation Fund': [8],
    'QDII fund': [8],
    'Commodity Funds': [8],
    'Equity Fund': [8],
    'Index Fund': [8],
    'Closed-end funds': [8],
    'Insurance': [8],
    'Life Insurance': [8, 29],
    'Insurance': [8],
    'Liability': [8],
    'Bank financing': [8],
    'fixed income': [8],
    'Guaranteed income': [8],
    'Non-guaranteed income': [8],
    'Consumer loans': [8],
    'Internet banking': [8],
    'Direct Banking': [8],
    'Third party payment': [8],
    'Crowdfunding': [8],
    'Financial Services': [8],
    'Trust': [8],
    'Trust funds': [8],
    'Chattel trust': [8],
    'Real Estate Trust': [8],
    'Property Trust': [8],
    'Intellectual Property Trust': [8],
    'Stock Futures': [8],
    'Commodity Futures': [8],
    'Financial Futures': [8],
    'High Flow': [8],
    'High mobility': [8],
    'Liquid medium': [8],
    'Lower liquidity': [8],
    'Low liquidity': [8],
    'High profitability': [8],
    'Financial Supermarket': [8],
    'Financial Information': [8],
    'Lottery ticket': [1],
    'exchange rate': [8],
    'Antique collection': [2],
    'Tencent': [1],
    'game': [1],
    'Cards RPG':[1],
    'MMO':[1],
    'round':[1],
    'ARPG':[1],
    'MOBA':[1],
    'SLG (strategy)':[1],
    'Racing (RAC)':[1],
    'Flight Shooting':[1],
    'Shootout Shooting (STG)':[1],
    'music':[2],
    'Sports': [6],
    'Tower Defense':[1],
    'Business simulation':[1, 8],
    'Casual puzzle categories': [1],
    'Chess categories': [1, 4],
    'Parkour avoid class': [1, 6],
    'Comics Reading Tools': [12],
    'literature': [13, 10],
    "Read the operator's": [13],
    'read': [13],
    'Pirated content': [],
    'Listening to books category': [13],
    'Other Read': [13],
    'the film': [12],
    'Online booking class': [13, 10],
    'User Community': [7],
    'Vermicelli': [20],
    'Buy class': [14],
    'Doctors': [29],
    'Maternal and child population': [22, 9],
    'Prepare baby': [9],
    'Families with babies': [9],
    'Pregnant baby': [9],
    'Parenting stage': [9],
    'Families with big baby': [9],
    'Families with small babies': [9],
    'Youth House': [9],
    'Three Generations of children': [9],
    'Middle-aged home': [9],
    'Empty nesters': [9],
    'Property Industry new': [8],
    'Smart Shopping': [14],
    'Science and Technology': [10],
    'physical education': [6],
    'health': [27],
    'military': [26],
    'Animation': [12],
    'Olympic': [6],
    'entertainment': [12],
    'Children': [9],
    'advertising': [14],
    'microfilm': [12],
    'Funny': [12],
    'education': [10],
    'tourism': [18],
    'Fashion Arts': [2],
    'fashion': [2],
    'Mother': [25, 9],
    'car': [22],
    'game': [1],
    'Trailer': [12],
    'life': [3],
    'the film': [12],
    'TV series': [12],
    'Science and Technology': [10],
    'Documentary': [10],
    'Arts': [2],
    'Talk show': [12],
    'Finance and economics': [8, 10],
    'Information': [12],
    'music': [2],
    'Air Travel': [18],
    'aviation': [6],
    'Airline type': [18],
    'Full Service Airlines': [18],
    'Low Cost Airlines': [18],
    'Regional Aviation': [18],
    'High-end business aviation': [18],
    'General Aviation': [18],
    'Aviation Integrated Services': [18],
    'Aeronautical Information Service': [18],
    'Airline Alliances': [18],
    'airport': [18],
    'Flight area': [18],
    'Chinese mainland (Airlines)': [18],
    'Southeast Asia (aviation)': [18],
    'Japan and South Korea (Air)': [18],
    'Europe, the United States and Macao (aviation)': [18],
    'Hong Kong, Macao and Taiwan (aviation)': [18],
    'travel': [18],
    'Purpose of travel': [18],
    'Travel Travel': [18],
    'Overseas travel': [18],
    'Domestic travel': [18],
    'Integrated tourism': [18],
    'Tour around': [18],
    'Travel Information': [18],
    'Free exercise': [10],
    'Travel preferences': [18],
    'Share Tour': [18],
    'Destination Region': [18],
    'Southeast Asia (Travel)': [18],
    'Japan and South Korea (Travel)': [18],
    'Europe, the United States and Macao (Travel)': [18],
    'Hong Kong, Macao and Taiwan (Travel)': [18],
    'Booking channels': [18],
    'Integrated ticket reservation': [18],
    'Book hotel complex': [18],
    'High-speed rail train reservation': [18],
    'Coach reservations': [18],
    'Integrated Air Travel Booking': [18],
    'Hotels': [18],
    'Hotel Type': [18],
    'High-end hotel': [18],
    'Hotel Chain': [18],
    'Non-standard accommodation': [18],
    'financial': [8],
    'Card Game': [1],
    'casual games': [1],
    'Puzzles': [1],
    'cosplay': [1, 7],
    'Sports Games': [6, 1],
    'Table Games': [1],
    'action games': [1],
    'strategy game': [1],
    'Simulators': [1],
    'Adventure Game': [1],
    'Chess game': [1],
    'Dice Game': [1],
    'Educational games': [1, 10],
    'Family Games': [1],
    'Music Games': [1, 2],
    'Racing games': [1],
    'Games': [1],
    'Word games': [1],
    'Xian Xia': [1],
    'navigation': [18],
    'reality show': [12],
    'Ninja': [6],
    'Martial arts': [6],
    'Domestic animation': [12],
    'Europe and Fantasy': [12],
    'fashion': [24],
    'Finance': [8],
    'Traditional securities brokerage': [8],
    'Lottery ticket': [8],
    'P2P net loan': [8],
    'Pay': [8],
    'Crowdfunding': [8],
    'Accounting': [8],
    'Third-party card management': [8],
    'Internet Banking': [8],
    'Financial Information': [8],
    'Consumer Finance': [8],
    'mobile bank': [8],
    'Traditional Insurance': [8],
    'Heritage Foundation': [8],
    'Direct Banking': [8],
    'Internet Securities': [8],
    'Bank Credit Card': [8],
    'Internet Insurance': [8]
}

In [8]:
list_types.get('Internet Insurance', [])
df_label_categories['category'] = df_label_categories['category'].apply(lambda x: list_types.get(x, []))

Combine `df_app_labels`, `df_label_categories` together

In [9]:
d = defaultdict(list)
for i, val in df_app_labels.iterrows():
    d[i] += df_label_categories.loc[val['label_id']]['category']

df_app_category = pd.DataFrame([(i, list(set(d[i]))) for i in d], columns=['app_id', 'cats']).set_index('app_id')

In [10]:
print len(df_gender_age_train), len(df_gender_age_test)

74645 112071


Now **df_app_labels**, **df_label_categories** are already combined and should not be used. New dataframe **df_app_category** allows to tell the category of the application.

In the end I need to have one big table, suitable for machine learning. This table would have:
 - device_id
 - **gender**
 - **age**
 - **group**
 - phone brand (`df_phone_brand_device_model`)
 - phone model (`df_phone_brand_device_model`)
 - number of events happend from this device (`df_events`)
 - activity at each hour (`df_events`)
 - activity at each day (`df_events`)
 - lat/lng (`df_events`)
 - all apps device used (`df_app_events`)
 - active apps device used (`df_app_events`)
 - categories of all apps (`df_app_category`)
 
Elements in bold would be only for training data.

Expect the number of rows to be ~200k. Will create each table separately.

In [11]:
%%time
app_events_data = defaultdict(list)
for event_id, val in df_app_events.iterrows():
    app_events_data[event_id].append((val['app_id'], val['is_active']))

CPU times: user 59min 59s, sys: 1min 19s, total: 1h 1min 19s
Wall time: 1h 3min 30s


In [12]:
def generate_data(df, is_test):
    data = []
    for device_id, val in df.iterrows():
        tmp = [device_id]
        if not is_test:
            tmp += [val['gender'], val['age'], val['group']]

        if device_id in df_phone_brand_device_model.index:
            d1 = df_phone_brand_device_model.loc[device_id]
            tmp += [d1['phone_brand'], d1['device_model']]
        else:
            tmp += ['other', 'other']

        tmp_events = df_events[df_events['device_id'] == device_id]
        tmp.append(len(tmp_events))

        if len(tmp_events):
            lng_lat_data, week_data, hour_data = [], [], []
            all_apps, all_active_apps = [], []
            for event_id, event_data in tmp_events.iterrows():
                lng_lat_data.append((event_data['longitude'], event_data['latitude']))
                week_data.append(event_data['week_day'])
                hour_data.append(event_data['hour'])

                for app_id, app_active in app_events_data[event_id]:
                    all_apps.append(app_id)
                    if app_active:
                        all_active_apps.append(app_id)

            tmp += [hour_data, week_data, lng_lat_data, all_apps, all_active_apps]
        else:
            tmp += [[], [], [], [], []]

        data.append(tmp)
    return data

In [13]:
%%time
data_train = generate_data(df_gender_age_train, False)
data_test = generate_data(df_gender_age_test, True)

CPU times: user 18min 54s, sys: 39.4 s, total: 19min 33s
Wall time: 19min 41s


In [14]:
common_cols = ['phone_brand', 'device_model', 'event_num', 'hours', 'days', 'location', 'all_apps', 'active_apps']
df_train = pd.DataFrame(data_train, columns=['device_id', 'gender', 'age', 'group'] + common_cols).set_index('device_id')
df_test = pd.DataFrame(data_test, columns=['device_id'] + common_cols).set_index('device_id')

In [15]:
def analyse_hours(arr):
    tmp = Counter(int(x) for x in arr)
    return [tmp[i] for i in xrange(24)]

def analyse_days(arr):
    tmp = Counter(int(x) for x in arr)
    return [tmp[i] for i in xrange(6)]

def analyse_num_apps(arr):
    return len(set(arr))

def analyse_categories(arr):
    categories = []
    for arr_id in set(arr):
        if arr_id in df_app_category.index:
            categories += df_app_category.loc[arr_id].cats
    
    tmp = Counter(categories)
    return [tmp[i] for i in xrange(1, len(category_types) + 1)] 

Analysing location is harder. I do the following:
 - get rid of locations close to (0, 0)
 - group the location based on coordinates
 - then group iteratively by combining a location based on the weighted average. This is done till the distance between the groups is below some threshold
 - after that select the most frequent coordinate and report the number of different coordinates

In [16]:
def group_locations(arr):
    if len(arr) < 2:
        return arr, False
    
    for i in xrange(len(arr)):
        for j in xrange(i + 1, len(arr)):
            loc1, n1 = arr[i]
            loc2, n2 = arr[j]
            if hypot(loc1[0] - loc2[0], loc1[1] - loc2[1]) < 40:
                # should be merged
                x1, y1 = loc1[0], loc1[1]
                x2, y2 = loc2[0], loc2[1]
                x_, y_ = (x1 * n1 + x2 * n2) / float(n1 + n2), (y1 * n1 + y2 * n2) / float(n1 + n2)
                
                arr_ = [((x_, y_), n1 + n2)]
                for i_ in xrange(len(arr)):
                    if i_ not in [i, j]:
                        arr_.append(arr[i_])
                        
                return arr_, True
    
    return arr, False

def group_locations_iteratively(cnt):
    arr, can_continue = [(val, cnt[val]) for val in cnt], True
    while can_continue:
        arr, can_continue = group_locations(arr)

    return arr

def analyse_location(arr):
    data = []
    for lng, lat in arr:
        if abs(lat) > 5 or abs(lng) > 5:
            data.append((int(lng * 10), int(lat * 10)))
    
    res = [((int(x * 10) / 100., int(y * 10) / 100.), n) for (x, y), n in group_locations_iteratively(Counter(data))]
    if res:
        return res[0][0], len(res)
    else:
        return (None, None), 0

In [17]:
def final_translation(df, is_test):
    data = []
    for device_id, val in df.iterrows():
        data_hours       = analyse_hours(val['hours'])
        data_days        = analyse_days(val['days'])
        data_loc         = analyse_location(val['location'])
        data_app_num_all = analyse_num_apps(val['all_apps'])
        data_app_num_act = analyse_num_apps(val['active_apps'])
        data_categories  = analyse_categories(val['all_apps'])
        
        tmp = [data_loc[0][0], data_loc[0][1], data_loc[1], data_app_num_all, data_app_num_act]
        line = [device_id, val['phone_brand'], val['device_model'], val['event_num']] + data_hours + data_days + tmp + data_categories
        
        if not is_test:
            line += [val['gender'], val['age'], val['group']]

        data.append(line)
    
    cols_hours = ['h_' + str(i) for i in xrange(24)]
    cols_days  = ['d_' + str(i) for i in xrange(6)]
    cols_cats  = ['c_' + str(i) for i in xrange(len(data_categories))]
    cols_tmp   = ['lng', 'lat', 'pos_n', 'app_num_all', 'app_num_active']
    cols = ['device_id', 'phone_brand', 'device_model', 'e_num'] + cols_hours + cols_days + cols_tmp + cols_cats
    
    if not is_test:
        cols += ['gender', 'age', 'group']
        
    return pd.DataFrame(data, columns=cols)

In [18]:
%%time
df_train_ = final_translation(df_train, False)
df_test_ = final_translation(df_test, True)

CPU times: user 8min 6s, sys: 10.5 s, total: 8min 16s
Wall time: 8min 12s


In [19]:
print df_train_.shape
print df_test_.shape

(74645, 71)
(112071, 68)


In [36]:
df_train_.to_csv('train.csv')
df_test_.to_csv('test.csv')

In [37]:
%%bash
ls

01_talking_data_exploration.ipynb
02_talking_data_preparation.ipynb
data
test.csv
train.csv
